<h1>Captura de Tweets</h1>

<h2>Se definen clases y funciones que permitan capturar tweets mediante la API TweePy</h2>

<p>En el directorio de trabajo debe haber una carpeta llamada **Config**. Dicha carpeta debe obtener dos archivos:</p>
<ul>
    <li>KeyWords.txt: un archivo de texto plano que contendrá por cada línea los términos a buscar en los tweets.</li>
    <li>config.json: {
        "API_KEY": "...",
        "API_SECRET": "...",
        "ACCESS_TOKEN": "...",
        "ACCESS_TOKEN_SECRET": "...",
        "TERMS_FILE": "./Config/KeyWords.txt",
        "STORAGE_PATH": "./Tweets/"
    }</li>
</ul>
<p>También debe contener una carpeta llamada Tweets para almacenar los archivos CSV que contendrán los tweets capturados</p>

In [39]:
import tweepy
import json
import logging 
import pandas as pd

In [40]:
class Authentication:
    
    """
    Función que me permite recorrer un archivo JSON, en donde iremos recorriendo línea por línea para obtener
    el contenido de cada atributo y devolverlos como resultado
    """
    def read_config_file(self, filename):
        with open(filename, "r") as line:
            json_content = line.read()
        attributes = json.loads(json_content)
        API_KEY = attributes["API_KEY"]
        API_SECRET = attributes["API_SECRET"]
        ACCESS_TOKEN = attributes["ACCESS_TOKEN"]
        ACCESS_TOKEN_SECRET = attributes["ACCESS_TOKEN_SECRET"]
        TERMS_FILE = attributes["TERMS_FILE"]
        TWEETS_STORAGE_PATH = attributes["TWEETS_STORAGE_PATH"]
        return API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET, TERMS_FILE, TWEETS_STORAGE_PATH
    
    def connect_api(self, API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET):
        auth = tweepy.OAuthHandler(API_KEY,API_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        api = tweepy.API(auth)
        return api

In [41]:
def get_authentication(configFile):
    
    """
    Creo la instancia de autenticación
    """
    auth = Authentication()
    
    """
    Formateo el log
    """
    logging.basicConfig(
        format = '%(levelname)s: %(asctime)s - %(message)s',
        datefmt = '%m/%d/%Y %I:%M:%S %p',
        level = logging.INFO)
    
    logging.info("Obteniendo información del archivo config.json")
    API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET, TERMS_FILE, TWEETS_STORAGE_PATH = auth.read_config_file(configFile)

    logging.info("Obteniendo autorización")
    api = auth.connect_api(API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

    logging.info("Autorización generada con éxito")
    logging.info("Parámetros leídos con éxito")

    filter_terms = []
    with open(TERMS_FILE) as f:
        for term in f:
            filter_terms.append(term.strip())
    logging.info("Los términos a buscar en los tweets son " + str(filter_terms))

    return api, filter_terms, TWEETS_STORAGE_PATH
    

In [42]:
def get_tweets(tw_api, tw_quantity, tweets_storage_path):
    
    for term in filter_terms:
        id, created_at, name, followers_count, full_text, favorite_count, retweet_count =[],[],[],[],[],[],[]
        file = tweets_storage_path + "Tweets_"+str(term)+".csv"
        logging.info("Buscando tweets para " + str(term))
        tweets = [status for status in tweepy.Cursor(tw_api.search, q=term, tweet_mode='extended', lan='es').items(tw_quantity)]
        logging.info("Cantidad de tweets " + str(len(tweets)))
        for tweet in tweets:
            id.append(tweet.id)
            created_at.append(tweet.created_at)
            name.append(tweet.user.screen_name)
            followers_count.append(tweet.user.followers_count)
            full_text.append(tweet.full_text)
            favorite_count.append(tweet.favorite_count)
            retweet_count.append(tweet.retweet_count)
            df = pd.DataFrame( {'id': id,
                                'created_at': created_at,
                                'name': name,
                                'followers_count': followers_count,
                                'full_text': full_text,
                                'favorite_count': favorite_count,
                                'retweet_count': retweet_count,
                                'flag': term
                                })
        df.to_csv(file)
        logging.info("Archivo generado: " + file)

In [43]:
api, filter_terms, TWEETS_STORAGE_PATH = get_authentication("./Config/config.json")
get_tweets(api, 600, TWEETS_STORAGE_PATH)

INFO: 04/03/2019 04:02:32 PM - Obteniendo información del archivo config.json
INFO: 04/03/2019 04:02:32 PM - Obteniendo autorización
INFO: 04/03/2019 04:02:33 PM - Autorización generada con éxito
INFO: 04/03/2019 04:02:33 PM - Parámetros leídos con éxito
INFO: 04/03/2019 04:02:33 PM - Los términos a buscar en los tweets son ['Kirchner', 'Lavagna', 'Macri']
INFO: 04/03/2019 04:02:33 PM - Buscando tweets para Kirchner
INFO: 04/03/2019 04:03:03 PM - Cantidad de tweets 600
INFO: 04/03/2019 04:03:05 PM - Archivo generado: ./Tweets/Tweets_Kirchner.csv
INFO: 04/03/2019 04:03:05 PM - Buscando tweets para Lavagna
INFO: 04/03/2019 04:03:41 PM - Cantidad de tweets 600
INFO: 04/03/2019 04:03:42 PM - Archivo generado: ./Tweets/Tweets_Lavagna.csv
INFO: 04/03/2019 04:03:42 PM - Buscando tweets para Macri
INFO: 04/03/2019 04:04:12 PM - Cantidad de tweets 600
INFO: 04/03/2019 04:04:12 PM - Archivo generado: ./Tweets/Tweets_Macri.csv
